This notebook is intended only as an example of how to read a bag file.

In order to run this notebook you will need ROS [Kinetic Kame](http://wiki.ros.org/kinetic) and Python 2.7.

The whole dataset can be downloaded (6.6 GB) [here](https://drive.switch.ch/index.php/s/1Q0zN0XDzyRxug4); in this example we will use only `1.bag`.

In [1]:
import os
import sys
from multiprocessing import Pool

import pandas as pd
import rosbag
import tqdm as tqdm

from direct_controller import Controller
from global_parameters import *
from utils import *

This dictionary is present in `global_parameters.py` but reported here for ease of reading

In [2]:
bag_file_path_example = {
    "1": "../bagfiles/train/",
}

We will read the information from a bag file and save it as dataset.

In [3]:
file_name = "1.bag"
path = bag_file_path_example[file_name[:-4]]
print(path+file_name)
bag = rosbag.Bag(path + file_name)

../bagfiles/train/1.bag


With `bag` loaded we can start to extract information from the topics. In the script `dataset_generator.py` we would call `get_bag_data_pandas()` and pass the `bag` variable and the model type. Here we will on show how the function works. 

---
We will create a single dataset for the model of approach 2

In [4]:
bt_id = [] # list of ids
bt_v = [] # list of values

In [5]:
# we extract the optitrack twist information
for topic, beboptw, t in bag.read_messages(topics=['/bebop/mocap_odom']): # for each message(beboptw) in the topic
    secs = t.secs 
    nsecs = t.nsecs
    bt_id.append(time_conversion_to_nano(secs, nsecs)) # time.secs and time.nsecs are summed (as nano sec) 
                                                       # and used as id.
    twist_dict = (lambda x: {'t_x': x.x,
                             't_y': x.y})(beboptw.twist.twist.linear) # We extract only the information that 
                                                                      # we are interested in
    bt_v.append(twist_dict) # we append it to the value list

In [6]:
bebop_twist_df = pd.DataFrame(data=bt_v, index=bt_id, columns=bt_v[0].keys()) # with the bt_id and bt_v lists we
                                                                              # create a Pandas DataFrame

We repeat this process for all the topics that we are interested in

In [7]:
# extract data from "/bebop/odom"
odom_id = []
odom_v = []
for topic, msg, t in bag.read_messages(topics=['/bebop/odom']):
    vel = msg.twist
    results = [vel.twist.linear.x, vel.twist.linear.y]
    secs = t.secs
    nsecs = t.nsecs
    odom_id.append(time_conversion_to_nano(secs, nsecs))
    vel_dict = (lambda x: {'x_vel': x[0],
                           'y_vel': x[1]})(results)
    odom_v.append(vel_dict)
odom_df = pd.DataFrame(data=odom_v, index=odom_id, columns=odom_v[0].keys())

# extract data from "/bebop/image_raw/compressed"
c_id = []
c_v = []
for topic, image_frame, t in bag.read_messages(topics=['/bebop/image_raw/compressed']):
    secs = t.secs
    nsecs = t.nsecs
    c_id.append(time_conversion_to_nano(secs, nsecs))
    img = jpeg2np(image_frame.data, (image_width, image_height))
    camera_frame = (lambda x: {'vid': x})(img)
    c_v.append(camera_frame)
camera_df = pd.DataFrame(data=c_v, index=c_id, columns=c_v[0].keys())

# extract data from "/optitrack/head"
h_id = []
h_v = []
for topic, hat, t in bag.read_messages(topics=['/optitrack/head']):
    secs = t.secs
    nsecs = t.nsecs
    h_id.append(time_conversion_to_nano(secs, nsecs))
    pos_rot_dict = (lambda x, y: {'h_pos_x': x.x,
                                  'h_pos_y': x.y,
                                  'h_pos_z': x.z,
                                  'h_rot_w': y.w,
                                  'h_rot_x': y.x,
                                  'h_rot_y': y.y,
                                  'h_rot_z': y.z})(hat.pose.position, hat.pose.orientation)
    h_v.append(pos_rot_dict)
head_df = pd.DataFrame(data=h_v, index=h_id, columns=h_v[0].keys())

# extract data from "/optitrack/bebop"
b_id = []
b_v = []
for topic, bebop, t in bag.read_messages(topics=['/optitrack/bebop']):
    secs = t.secs
    nsecs = t.nsecs
    b_id.append(time_conversion_to_nano(secs, nsecs))
    pos_rot_dict = (lambda x, y: {'b_pos_x': x.x,
                                  'b_pos_y': x.y,
                                  'b_pos_z': x.z,
                                  'b_rot_w': y.w,
                                  'b_rot_x': y.x,
                                  'b_rot_y': y.y,
                                  'b_rot_z': y.z})(bebop.pose.position, bebop.pose.orientation)
    b_v.append(pos_rot_dict)
bebop_df = pd.DataFrame(data=b_v, index=b_id, columns=b_v[0].keys())

then we close the bag file and create a dictionary with all the extracted dataframes

In [8]:
bag.close()
resulting_dictionary = {'head_df': head_df, 'bebop_df': bebop_df, 'odom_df': odom_df,\
                        'camera_df': camera_df, 'bebop_twist_df': bebop_twist_df}

In the repository script we return this dictionary and use it as a parameter for another method call: `processing()`

In [9]:
f = file_name
data_id = f
# we select the ids of each dataframe, NB id == timestamp in nanosec
camera_t = resulting_dictionary["camera_df"].index.values
bebop_t = resulting_dictionary["bebop_df"].index.values
head_t = resulting_dictionary["head_df"].index.values
odom_t = resulting_dictionary["odom_df"].index.values
bebop_twist_t = resulting_dictionary["bebop_twist_df"].index.values
print(camera_t)

[1524134860684665704 1524134860685877347 1524134860686668596 ...
 1524134972471268497 1524134972498969542 1524134972531661089]


In [10]:
# As explained in the README, each bag file is "cut" to avoid useless sections.
bag_end_cut_example = {
    "1": 3150
}
bag_start_cut_example = {
    "1": 0
}
max_ = bag_end_cut_example[f[:-4]]
min_ = bag_start_cut_example[f[:-4]]
data_vec = []

# We initialize the hand-programmed controller
d_ctrl = Controller()

Now we iterate on the dataframes rows  in order to:
* synchronize the rows
* compute new information

In [11]:
for i in tqdm.tqdm(range(min_, max_), desc="processing data " + str(data_id)):
    
    # we find the nearest datapoint to the i-th datapoint of the camera feed
    b_id = find_nearest(bebop_t, camera_t[i])
    h_id = find_nearest(head_t, camera_t[i])
    odom_id = find_nearest(odom_t, camera_t[i])
    
    # with the new ids we extract the nearest rows in each dataframe
    head_pose = resulting_dictionary["head_df"].iloc[h_id]
    bebop_pose = resulting_dictionary["bebop_df"].iloc[b_id]
    odom_info = resulting_dictionary["odom_df"].iloc[odom_id]
    
    # we also extract the video feed
    img = resulting_dictionary["camera_df"].iloc[i].values[0]
    
    # we change the frame of reference of the user's head's pose from World to Drone (bebop). the result is a 4x4 
    # rototranslation matrix
    b_t_h = change_frame_reference(bebop_pose, head_pose)
    
    # we extract the visual odometry velocities
    vel_x = odom_info.x_vel
    vel_y = odom_info.y_vel

    # we extract the yaw angle of the drone and user's head
    quaternion_bebop = bebop_pose[['b_rot_x', 'b_rot_y', 'b_rot_z', 'b_rot_w']].values
    _, _, bebop_yaw = quat_to_eul(quaternion_bebop)
    quaternion_head = head_pose[['h_rot_x', 'h_rot_y', 'h_rot_z', 'h_rot_w']].values
    _, _, head_yaw = quat_to_eul(quaternion_head)

    # we compute the relative yaw of the user's head wrt the drone
    relative_yaw = (head_yaw - bebop_yaw - np.pi)
    if relative_yaw < -np.pi:
        relative_yaw += 2 * np.pi
        
    # we extract the user (target) position vector from the matrix 
    target_position = b_t_h[:-1, -1:].T[0]
    target = (target_position[0], target_position[1], target_position[2], relative_yaw)
    
    # we append the selected variables to a list
    data_vec.append((img, target, np.asarray([vel_x, vel_y])))

processing data 1.bag: 100%|██████████| 3150/3150 [00:12<00:00, 247.16it/s]


`data_vec` is returned and passed to another method in order to save it as pickle. Below we directly save it as `example.pickle`

In [12]:
# we would use 
# f_pickle_ = f[:-4] + ".pickle"
f_pickle_ = "example.pickle"
df = pd.DataFrame(list(data_vec))
df.to_pickle("../dataset/" + f_pickle_)